<a href="https://colab.research.google.com/github/TarekBellalouna/la7winta/blob/ecommerce/Systeme_de_recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install scikit-surprise

In [20]:
!pip install colabcode

In [21]:
!pip install fastapi

In [22]:
from surprise import Dataset, Reader
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic,  KNNWithMeans, KNNBaseline
from surprise.model_selection import KFold
from surprise import Reader
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
import matplotlib.pyplot as plt
import seaborn as sns
from surprise.model_selection import GridSearchCV
from collections import defaultdict
from colabcode import ColabCode
from fastapi import FastAPI
import json

In [23]:
import pandas as pd

data = {'Profils':  ['Mohamed Youssef', 'Mohamed Yassine','Amr Bellalouna','Hamza Razgallah','Nour Zemni','Anis Kraiem','Akram Thabet','Ela Debichi','Selim ben Aich','Bassem Jadoui','Karim mannai', 'Mohamed Amara','Slim Jemai','Mehdi Hantous','Skander Bachta','Ala ben Ali','Bilel Moussa','Iheb Guezguez','Mounir Maalej','Zied Maalej'],
        'Products': ['Short','T-Shirt','Shirt','Pants','Glasses','Watch','Shoes','Sneakers','Jeans','Sweats','Coats','Socks','Jacket','Veste','Robes','Jeans Armani','Shirt zara','Sneakers Nike','Sweat Adidas','Jeans Lee Cooper'],
        'Appreciation':[4,1,3,4,3,5,2,3,3,1,2,5,5,3,1,4,3,1,2,2]
        }

df = pd.DataFrame (data, columns = ['Profils','Products','Appreciation'])

In [24]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

In [25]:
data = Dataset.load_from_df(df,reader)

In [26]:
kf = KFold(n_splits=3)
algo = KNNBasic()
best_algo = None
best_rmse = 1000.0
best_pred = None
for trainset, testset in kf.split(data):
    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)
    if rmse < best_rmse:
        best_algo = algo
        best_pred = predictions

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.5249
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3447
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1744


In [28]:
def get_best_products(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    best_products = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        best_products[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in best_products.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        best_products[uid] = user_ratings[:n]

    return best_products

In [27]:
best_pred

[Prediction(uid='Nour Zemni', iid='Glasses', r_ui=3.0, est=2.7857142857142856, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='Mohamed Amara', iid='Socks', r_ui=5.0, est=2.7857142857142856, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='Mohamed Yassine', iid='T-Shirt', r_ui=1.0, est=2.7857142857142856, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='Amr Bellalouna', iid='Shirt', r_ui=3.0, est=2.7857142857142856, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='Ela Debichi', iid='Sneakers', r_ui=3.0, est=2.7857142857142856, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid='Bilel Moussa', iid='Shirt zara', r_ui=3.0, est=2.7857142857142856, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})]

In [29]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

best_products = get_best_products(predictions, n=10)

In [30]:
best_products

defaultdict(list,
            {'Akram Thabet': [('Robes', 2.7857142857142856),
              ('Jacket', 2.7857142857142856),
              ('Veste', 2.7857142857142856),
              ('Coats', 2.7857142857142856),
              ('Jeans Lee Cooper', 2.7857142857142856),
              ('Short', 2.7857142857142856),
              ('Watch', 2.7857142857142856),
              ('Jeans', 2.7857142857142856),
              ('Pants', 2.7857142857142856),
              ('Jeans Armani', 2.7857142857142856)],
             'Ala ben Ali': [('Robes', 2.7857142857142856),
              ('Jacket', 2.7857142857142856),
              ('Veste', 2.7857142857142856),
              ('Coats', 2.7857142857142856),
              ('Jeans Lee Cooper', 2.7857142857142856),
              ('Short', 2.7857142857142856),
              ('Watch', 2.7857142857142856),
              ('Shoes', 2.7857142857142856),
              ('Jeans', 2.7857142857142856),
              ('Pants', 2.7857142857142856)],
             'Anis

In [31]:
cc = ColabCode(port=4002, code=False)

In [32]:
app = FastAPI()

@app.get("/recommandation")
async def read_root():
  return {json.dumps(best_products)}

In [ ]:
cc.run_app(app=app)